In [1]:
import sys
import os

# Path to the project root (one level above the notebooks folder)
project_root = os.path.abspath("..")

# Add to Python path if not already present
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added:", project_root)

Project root added: /home/zervaki/Thesis_New


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# MVPC (original)
from mvpc.mvpc_pipeline import MVPC
from mvpc.ci_tests.gauss_permc import gauss_ci_td, gauss_ci_permc
from mvpc.ci_tests.gauss_drw import gauss_ci_drw

# MVPC Oracle
from mvpc.mvpc_oracle import MVPC_Oracle

# Missingness generation (synthetic)
from data.synthetic_data_generation.missingness_synthetic import (
    create_mar_ind, create_mnar_ind,
    generate_missing_values, generate_mcar_reference
)

# DAG utilities
from data.synthetic_data_generation.dag_and_data import (
    detect_colliders, detect_collider_parents
)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


In [3]:
df = pd.read_csv("../data/processed_mimic/processed_mimic_24h_labs_demographics.csv")

mvpc_vars = [
    "Sodium", "Potassium", "Chloride", "Creatinine", "Urea Nitrogen",
    "Hematocrit", "Hemoglobin", "WBC", "Platelet Count", "Glucose",
    "anchor_age", "length_of_stay_hours",
]

X_real = df[mvpc_vars].to_numpy()

scaler = StandardScaler()
X_complete = scaler.fit_transform(X_real)

print("X_complete shape:", X_complete.shape)


X_complete shape: (234, 12)


In [4]:
num_var = len(mvpc_vars)
idx = {v: i for i, v in enumerate(mvpc_vars)}

adj_true = np.zeros((num_var, num_var))

# Collider 1
adj_true[idx["Sodium"], idx["Creatinine"]] = 1
adj_true[idx["Potassium"], idx["Creatinine"]] = 1

# Collider 2
adj_true[idx["Chloride"], idx["Urea Nitrogen"]] = 1
adj_true[idx["Potassium"], idx["Urea Nitrogen"]] = 1

# Collider 3
adj_true[idx["Sodium"], idx["Glucose"]] = 1
adj_true[idx["Chloride"], idx["Glucose"]] = 1

# Collider 4
adj_true[idx["Creatinine"], idx["Hemoglobin"]] = 1
adj_true[idx["Glucose"], idx["Hemoglobin"]] = 1

# Collider 5
adj_true[idx["Glucose"], idx["WBC"]] = 1
adj_true[idx["Urea Nitrogen"], idx["WBC"]] = 1

# Collider 6
adj_true[idx["Hemoglobin"], idx["Platelet Count"]] = 1
adj_true[idx["Hematocrit"], idx["Platelet Count"]] = 1

# Collider 7
adj_true[idx["Hemoglobin"], idx["length_of_stay_hours"]] = 1
adj_true[idx["WBC"], idx["length_of_stay_hours"]] = 1
adj_true[idx["anchor_age"], idx["length_of_stay_hours"]] = 1

adj_true


array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [5]:
colliders = detect_colliders(adj_true)
collider_parents = detect_collider_parents(adj_true, colliders)

print("Colliders:", colliders)
print("Collider parents:", collider_parents)


Colliders: [3, 4, 6, 7, 8, 9, 11]
Collider parents: [[np.int64(0), np.int64(1)], [np.int64(1), np.int64(2)], [np.int64(3), np.int64(9)], [np.int64(4), np.int64(9)], [np.int64(5), np.int64(6)], [np.int64(0), np.int64(2)], [np.int64(6), np.int64(7), np.int64(10)]]


In [6]:
seed = 42

ms_mar, prt_ms_mar = create_mar_ind(
    colliders, collider_parents, num_var,
    num_extra_e=3, num_m=6, seed=seed
)

ms_mnar, prt_ms_mnar = create_mnar_ind(
    colliders, collider_parents, num_var,
    num_extra_e=3, num_m=6, seed=seed
)


In [7]:
def build_prt_m_from_ms(ms, prt_ms):
    prt_dict = {}
    for m, p in zip(ms, prt_ms):
        prt_dict.setdefault(m, []).append(p)
    m_inds = sorted(prt_dict.keys())
    return {"m": m_inds, "prt": prt_dict}


In [8]:
X_mar = generate_missing_values(
    X_complete, ms_mar, prt_ms_mar,
    p_missing_h=0.9, p_missing_l=0.1, seed=seed
)

X_mcar = generate_mcar_reference(
    X_complete, X_mar, ms_mar, seed=seed
)

X_mnar = generate_missing_values(
    X_complete, ms_mnar, prt_ms_mnar,
    p_missing_h=0.9, p_missing_l=0.1, seed=seed
)


In [9]:
prt_m_mar  = build_prt_m_from_ms(ms_mar,  prt_ms_mar)
prt_m_mnar = build_prt_m_from_ms(ms_mnar, prt_ms_mnar)
prt_m_mcar = prt_m_mar


In [10]:
def shd_directed(G_est, G_true):
    return np.sum(G_est != G_true)

def f1_directed(G_est, G_true):
    est = G_est.flatten()
    true = G_true.flatten()

    TP = np.sum((est == 1) & (true == 1))
    FP = np.sum((est == 1) & (true == 0))
    FN = np.sum((est == 0) & (true == 1))

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall    = TP / (TP + FN) if TP + FN > 0 else 0
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


In [11]:
def run_mvpc_oracle_all_methods(X_m, adj_true, prt_m):
    results = {}

    mvpc_td = MVPC_Oracle(gauss_ci_td, gauss_ci_td)
    G_td = mvpc_td.run(X_m, prt_m)["G_corrected"]
    results["TD"] = {"shd": shd_directed(G_td, adj_true), "f1": f1_directed(G_td, adj_true)}

    mvpc_permc = MVPC_Oracle(gauss_ci_td, gauss_ci_permc)
    G_permc = mvpc_permc.run(X_m, prt_m)["G_corrected"]
    results["PERMC"] = {"shd": shd_directed(G_permc, adj_true), "f1": f1_directed(G_permc, adj_true)}

    mvpc_drw = MVPC_Oracle(gauss_ci_td, gauss_ci_drw)
    G_drw = mvpc_drw.run(X_m, prt_m)["G_corrected"]
    results["DRW"] = {"shd": shd_directed(G_drw, adj_true), "f1": f1_directed(G_drw, adj_true)}

    return results


In [12]:
results_oracle = {
    "MAR":  run_mvpc_oracle_all_methods(X_mar,  adj_true, prt_m_mar),
    "MNAR": run_mvpc_oracle_all_methods(X_mnar, adj_true, prt_m_mnar),
    "MCAR": run_mvpc_oracle_all_methods(X_mcar, adj_true, prt_m_mcar),
}

results_oracle


{'MAR': {'TD': {'shd': np.int64(29), 'f1': np.float64(0.06451612903225808)},
  'PERMC': {'shd': np.int64(29), 'f1': np.float64(0.06451612903225808)},
  'DRW': {'shd': np.int64(29), 'f1': np.float64(0.06451612903225808)}},
 'MNAR': {'TD': {'shd': np.int64(27), 'f1': np.float64(0.1818181818181818)},
  'PERMC': {'shd': np.int64(27), 'f1': np.float64(0.1818181818181818)},
  'DRW': {'shd': np.int64(27), 'f1': np.float64(0.1818181818181818)}},
 'MCAR': {'TD': {'shd': np.int64(23), 'f1': np.float64(0.08)},
  'PERMC': {'shd': np.int64(23), 'f1': np.float64(0.08)},
  'DRW': {'shd': np.int64(23), 'f1': np.float64(0.08)}}}

In [13]:
rows = []
for mode, methods in results_oracle.items():
    for method, metrics in methods.items():
        rows.append({
            "mode": mode,
            "method": method,
            "shd": float(metrics["shd"]),
            "f1": float(metrics["f1"]),
        })

df_oracle_results = pd.DataFrame(rows)
df_oracle_results


,mode,method,shd,f1
0,MAR,TD,29.0,0.064516
1,MAR,PERMC,29.0,0.064516
2,MAR,DRW,29.0,0.064516
3,MNAR,TD,27.0,0.181818
4,MNAR,PERMC,27.0,0.181818
5,MNAR,DRW,27.0,0.181818
6,MCAR,TD,23.0,0.080000
7,MCAR,PERMC,23.0,0.080000
8,MCAR,DRW,23.0,0.080000
